In [1]:
import numpy as np
import pandas as pd
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *


In [2]:
df_stocks = pd.read_pickle('/home/shan/stock_market/Stock_Market_Prediction/Data/Pickled_data.pkl')
print(df_stocks.info())
df_stocks.head()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3653 entries, 2007-01-01 to 2016-12-31
Freq: D
Data columns (total 3 columns):
close        3653 non-null float64
adj close    3653 non-null float64
articles     3653 non-null object
dtypes: float64(2), object(1)
memory usage: 114.2+ KB
None


,close,adj close,articles
2007-01-01,12469.971875,12469.971875,. What Sticks from '06. Somalia Orders Islamis...
2007-01-02,12472.245703,12472.245703,. Heart Health: Vitamin Does Not Prevent Death...
2007-01-03,12474.519531,12474.519531,. Google Answer to Filling Jobs Is an Algorith...
2007-01-04,12480.690430,12480.690430,. Helping Make the Shift From Combat to Commer...
2007-01-05,12398.009766,12398.009766,. Rise in Ethanol Raises Concerns About Corn a...


In [3]:
df_stocks['prices'] = df_stocks['adj close'].apply(np.int64)
print(df_stocks.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3653 entries, 2007-01-01 to 2016-12-31
Freq: D
Data columns (total 4 columns):
close        3653 non-null float64
adj close    3653 non-null float64
articles     3653 non-null object
prices       3653 non-null int64
dtypes: float64(2), int64(1), object(1)
memory usage: 142.7+ KB
None


In [4]:
# selecting the prices and articles
df_stocks = df_stocks[['prices', 'articles']]
df_stocks['articles'] = df_stocks['articles'].map(lambda x: x.lstrip('.-'))
#print(df_stocks.head())


In [13]:
df = df_stocks[['prices']].copy()
#print(df.head())
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''
df.head()
df_stocks.articles.dtype

dtype('O')

In [49]:
#Now starts score prediction for given sentence.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import nltk
#nltk.download('vader_lexicon')
import unicodedata
saf = SentimentIntensityAnalyzer()   #df_stocks.T provides the transpose of data.
T_df_stocks=df_stocks.T
#print(T_df_stocks.head())

for date, row in df_stocks.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df_stocks.loc[date, 'articles'])
        #print(sentence)
        ss = saf.polarity_scores(sentence)
        df.at(date, 'compound', ss['compound'])
        df.at(date, 'neg', ss['neg'])
        df.at(date, 'neu', ss['neu'])
        df.at(date, 'pos', ss['pos'])
    except TypeError:
        #print (df_stocks.loc[date, 'articles'].head())
        date


In [50]:
df.head()

,prices,compound,neg,neu,pos
2007-01-01,12469,-0.9814,0.159,0.749,0.093
2007-01-02,12472,-0.8179,0.114,0.787,0.099
2007-01-03,12474,-0.9993,0.198,0.737,0.065
2007-01-04,12480,-0.9982,0.131,0.806,0.062
2007-01-05,12398,-0.9901,0.124,0.794,0.082


In [93]:
#sentence = 'paris shootout police officer suspected guman dead'
#sentence ='ohh no icici deal '
#from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import unicodedata
#sid = SentimentIntensityAnalyzer()
#ss = sid.polarity_scores(sentence)
#ss

{'neg': 0.423, 'neu': 0.577, 'pos': 0.0, 'compound': -0.296}

In [97]:
train_start_date = '2007-01-01'
train_end_date = '2014-12-31'
test_start_date = '2015-01-01'
test_end_date = '2016-12-31'
train = df.loc[train_start_date : train_end_date]
test = df.loc[test_start_date:test_end_date]
print(train.head())
print(test.head())

            prices compound    neg    neu    pos
2007-01-01   12469  -0.9814  0.159  0.749  0.093
2007-01-02   12472  -0.8179  0.114  0.787  0.099
2007-01-03   12474  -0.9993  0.198  0.737  0.065
2007-01-04   12480  -0.9982  0.131  0.806  0.062
2007-01-05   12398  -0.9901  0.124  0.794  0.082
            prices compound    neg    neu    pos
2015-01-01   17828  -0.9299  0.138   0.74  0.122
2015-01-02   17832  -0.9775  0.149  0.759  0.093
2015-01-03   17722  -0.9737  0.131  0.787  0.083
2015-01-04   17612  -0.9971   0.22  0.716  0.064
2015-01-05   17501  -0.9905  0.156  0.767  0.077


In [101]:
sentiment_score_list = []
for date, row in train.T.iteritems():
    #sentiment_score = np.asarray([df.loc[date, 'compound'],df.loc[date, 'neg'],df.loc[date, 'neu'],df.loc[date, 'pos']])
    sentiment_score = np.asarray([df.loc[date, 'neg'],df.loc[date, 'pos']])
    sentiment_score_list.append(sentiment_score)
numpy_df_train = np.asarray(sentiment_score_list)
numpy_df_train

array([[0.159, 0.093],
       [0.114, 0.099],
       [0.198, 0.065],
       ...,
       [0.171, 0.075],
       [0.16 , 0.14 ],
       [0.198, 0.135]])

In [102]:
sentiment_score_list = []
for date, row in test.T.iteritems():
    #sentiment_score = np.asarray([df.loc[date, 'compound'],df.loc[date, 'neg'],df.loc[date, 'neu'],df.loc[date, 'pos']])
    sentiment_score = np.asarray([df.loc[date, 'neg'],df.loc[date, 'pos']])
    sentiment_score_list.append(sentiment_score)
numpy_df_test = np.asarray(sentiment_score_list)
numpy_df_test

array([[0.138, 0.122],
       [0.149, 0.093],
       [0.131, 0.083],
       ...,
       [0.138, 0.097],
       [0.168, 0.098],
       [0.173, 0.161]])

In [103]:
y_train = pd.DataFrame(train['prices'])
y_test = pd.DataFrame(test['prices'])
print(y_train)
print(y_test)

            prices
2007-01-01   12469
2007-01-02   12472
2007-01-03   12474
2007-01-04   12480
2007-01-05   12398
2007-01-06   12406
2007-01-07   12414
2007-01-08   12423
2007-01-09   12416
2007-01-10   12442
2007-01-11   12514
2007-01-12   12556
2007-01-13   12562
2007-01-14   12569
2007-01-15   12575
2007-01-16   12582
2007-01-17   12577
2007-01-18   12567
2007-01-19   12565
2007-01-20   12536
2007-01-21   12506
2007-01-22   12477
2007-01-23   12533
2007-01-24   12621
2007-01-25   12502
2007-01-26   12487
2007-01-27   12488
2007-01-28   12489
2007-01-29   12490
2007-01-30   12523
...            ...
2014-12-02   17879
2014-12-03   17912
2014-12-04   17900
2014-12-05   17958
2014-12-06   17923
2014-12-07   17887
2014-12-08   17852
2014-12-09   17801
2014-12-10   17533
2014-12-11   17596
2014-12-12   17280
2014-12-13   17247
2014-12-14   17214
2014-12-15   17180
2014-12-16   17068
2014-12-17   17356
2014-12-18   17778
2014-12-19   17804
2014-12-20   17856
2014-12-21   17907
2014-12-22  

In [105]:

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix

rf = RandomForestRegressor()
rf.fit(numpy_df_train, y_train)
print (rf.feature_importances_)

/home/shan/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/shan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


[0.53718281 0.46281719]


In [107]:
from treeinterpreter import treeinterpreter as ti
prediction, bias, contributions = ti.predict(rf, numpy_df_test)


ModuleNotFoundError: No module named 'treeinterpreter'